**ISP CUSTOMER CHURN ANALYSIS DASHBOARD📊**

An Internet Service Provider (ISP) company is experiencing increasing customer churn due to network performance issues. Based on customer complaints and support tickets, major concerns include high packet loss and latency. The objective of this project is to analyze network performance across different areas of Bangalore and identify whether poor packet loss or high latency during peak hours is contributing to increased customer churn.

*The goal is to determine:*

1.Which areas have the highest churn rate

2.Whether packet loss or latency has a stronger impact on churn

3.Whether peak-hour network congestion is affecting customer retention

Based on the analysis, actionable recommendations will be provided to help 
the company reduce churn and improve service quality.

In [147]:
import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [131]:
d=pd.read_csv('C:/Users/vanashree g hegde/OneDrive/Documents/Data_Analysis/churn.csv')
d.head(5)

,customer_id,area,timestamp,packet_loss_percent,latency_ms,churn_flag
0,100000,Electronic City,2025-12-18 18:34:38.196244,1.996714,57.234714,0
1,100001,Yelahanka,2026-01-09 21:26:38.196244,1.552480,132.756087,0
2,100002,Yelahanka,2026-01-11 03:23:38.196244,3.079213,75.348695,0
3,100003,Whitefield,2026-01-07 17:33:38.196244,0.919122,49.496604,0
4,100004,Electronic City,2026-01-28 05:15:38.196244,1.741962,21.734395,0


In [132]:
d.isnull().sum()

customer_id              0
area                     0
timestamp                0
packet_loss_percent    229
latency_ms             245
churn_flag               0
dtype: int64

In [133]:
d['latency_ms']=d.groupby('area')['latency_ms'].transform(lambda x:x.fillna(x.median()))
d['packet_loss_percent']=d.groupby('area')['packet_loss_percent'].transform(lambda x:x.fillna(x.median()))

In [134]:
d.isnull().sum()

customer_id            0
area                   0
timestamp              0
packet_loss_percent    0
latency_ms             0
churn_flag             0
dtype: int64

In [135]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_id          8000 non-null   int64  
 1   area                 8000 non-null   object 
 2   timestamp            8000 non-null   object 
 3   packet_loss_percent  8000 non-null   float64
 4   latency_ms           8000 non-null   float64
 5   churn_flag           8000 non-null   int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 375.1+ KB


In [136]:
d['timestamp'] = pd.to_datetime(d['timestamp'], errors='coerce')
d['timestamp'].dtype

dtype('<M8[ns]')

In [137]:
d['churn_flag'].value_counts()

churn_flag
0    6597
1    1403
Name: count, dtype: int64

In [138]:
#churn rate per area
(d.groupby('area')['churn_flag'].mean()*100).round(2)

area
Electronic City    17.10
HSR Layout         20.07
Indiranagar        16.15
Koramangala        18.18
Rajajinagar        17.76
Whitefield         16.41
Yelahanka          16.97
Name: churn_flag, dtype: float64

In [139]:
#churn rate due to packetloss and latency
(d.groupby('churn_flag')[['packet_loss_percent','latency_ms']].mean()).round(2)

,packet_loss_percent,latency_ms
churn_flag,,
0,2.83,75.84
1,5.88,96.10


In [140]:
#packetloss and latency per area
d.groupby('area')[['packet_loss_percent','latency_ms']].mean()


,packet_loss_percent,latency_ms
area,,
Electronic City,3.645315,82.730940
HSR Layout,3.563982,83.611909
Indiranagar,2.995301,73.131397
Koramangala,3.697419,83.931882
Rajajinagar,3.071214,74.830416
Whitefield,3.820421,85.456469
Yelahanka,2.749401,72.301499


In [141]:
d['hour']=d['timestamp'].dt.hour

In [142]:
def peak_time(hour):
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 17:
        return "Afternoon"
    elif 17 <= hour < 22:
        return "Evening"
    else:
        return "Night"

d['time_of_day'] = d['hour'].apply(peak_time)


In [144]:
#packetloss and latency respective of time
d.groupby('time_of_day')[['packet_loss_percent','latency_ms']].mean()

,packet_loss_percent,latency_ms
time_of_day,,
Afternoon,2.838042,71.541936
Evening,5.011837,98.735636
Morning,3.428986,81.780672
Night,2.660481,71.024104


In [151]:
#connecting mssql server

con=pyodbc.connect(
    "DRIVER={ODBC Driver 17 for sql Server};"
    "SERVER=VANASHREE\MSSQL;"
    "DATABASE=project;"
    "Trusted_Connection=yes;"
)



In [152]:
eng=create_engine(
    "mssql+pyodbc://@VANASHREE\MSSQL/project?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

d.to_sql('data',eng)
print("Successfully uploaded")

Successfully uploaded
